# Spectral Calibration: S³ and T⁷ Benchmarks

**Objectif**: Déterminer si le biais observé sur K₇ (13 vs 14) est structurel ou un artefact du pipeline.

**Stratégie**: Appliquer le **même pipeline exact** que K₇ sur des espaces où λ₁ est connu analytiquement:

| Espace | Dimension | λ₁ exact | Pourquoi ce test |
|--------|-----------|----------|------------------|
| S³ | 3 | 3 | Sphère, spectre bien connu |
| T⁷ | 7 | 1 | Même dimension que K₇! |

**Question clé**: Si le pipeline donne λ₁ = 2.9 sur S³ (au lieu de 3), alors le −1 observé sur K₇ est un biais de discrétisation. Si le pipeline donne λ₁ ≈ 3 exact, alors le 13 sur K₇ est structurel.

---

## Théorie

### S³ (3-sphère de rayon 1)
- Spectre: λₙ = n(n+2) pour n = 0, 1, 2, ...
- λ₀ = 0, **λ₁ = 3**, λ₂ = 8, ...
- Multiplicité de λ₁: 4 (modes harmoniques sphériques)

### T⁷ (7-tore plat avec rayons unitaires)
- Spectre: λ = Σᵢ nᵢ² pour nᵢ ∈ ℤ
- λ₀ = 0, **λ₁ = 1** (un seul nᵢ = ±1)
- Multiplicité de λ₁: 14 (±1 sur chaque des 7 dimensions)

---

In [ ]:
# Cell 1: Setup - GPU/CPU detection

import numpy as np
import json
from datetime import datetime
import time
import os

# GPU detection
try:
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU_AVAILABLE = True
    print("✓ GPU available via CuPy")
    
    # Show GPU info
    device = cp.cuda.Device()
    props = cp.cuda.runtime.getDeviceProperties(device.id)
    print(f"  Device: {props['name'].decode()}")
    print(f"  Memory: {props['totalGlobalMem'] / 1e9:.1f} GB")
    
except ImportError:
    GPU_AVAILABLE = False
    from scipy.sparse import csr_matrix
    from scipy.sparse.linalg import eigsh
    cp = np
    print("✗ CuPy not available - using CPU")

# Matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

def clear_gpu():
    """Free GPU memory."""
    if GPU_AVAILABLE:
        cp.get_default_memory_pool().free_all_blocks()
        cp.get_default_pinned_memory_pool().free_all_blocks()

# Constants
LAMBDA1_S3 = 3.0      # Exact eigenvalue for S³
LAMBDA1_T7 = 1.0      # Exact eigenvalue for T⁷
H_STAR_K7 = 99        # For reference

print(f"\nExact eigenvalues:")
print(f"  S³: λ₁ = {LAMBDA1_S3}")
print(f"  T⁷: λ₁ = {LAMBDA1_T7}")
print(f"  K₇: λ₁ × H* ≈ 13 (empirical)")

In [ ]:
# Cell 2: S³ Sampling and Distance

def sample_S3(N: int, seed: int = 42) -> np.ndarray:
    """
    Sample N points uniformly on S³ (unit 3-sphere in R⁴).
    
    Method: Gaussian projection (exact uniform sampling).
    """
    rng = np.random.default_rng(seed)
    
    # Sample from 4D Gaussian
    points = rng.standard_normal((N, 4))
    
    # Project to unit sphere
    norms = np.linalg.norm(points, axis=1, keepdims=True)
    points = points / norms
    
    return points.astype(np.float32)


def geodesic_distance_S3(points: np.ndarray) -> np.ndarray:
    """
    Geodesic distance on S³.
    
    For unit quaternions: d(q₁, q₂) = 2 × arccos(|q₁ · q₂|)
    The factor 2 comes from the double cover S³ → SO(3).
    
    For S³ as unit sphere in R⁴: d(p₁, p₂) = arccos(p₁ · p₂)
    Range: [0, π]
    """
    N = points.shape[0]
    
    # Dot products
    dots = points @ points.T
    
    # Clip for numerical stability
    np.clip(dots, -1.0, 1.0, out=dots)
    
    # Geodesic distance = arccos(dot)
    D = np.arccos(dots)
    
    # Ensure diagonal is exactly 0
    np.fill_diagonal(D, 0.0)
    
    return D.astype(np.float32)


# Quick test
print("Testing S³ sampling...")
test_points = sample_S3(100, seed=42)
print(f"  Shape: {test_points.shape}")
print(f"  Norms: {np.linalg.norm(test_points, axis=1)[:5]} (should be 1.0)")

test_D = geodesic_distance_S3(test_points)
print(f"  Distance range: [{test_D.min():.4f}, {test_D.max():.4f}]")
print(f"  Expected max: π ≈ {np.pi:.4f}")
print("✓ S³ functions OK")

In [ ]:
# Cell 3: T⁷ Sampling and Distance

def sample_T7(N: int, seed: int = 42) -> np.ndarray:
    """
    Sample N points uniformly on T⁷ = [0, 2π)⁷.
    """
    rng = np.random.default_rng(seed)
    angles = rng.uniform(0, 2 * np.pi, size=(N, 7))
    return angles.astype(np.float32)


def geodesic_distance_T7(angles: np.ndarray) -> np.ndarray:
    """
    Geodesic (toric) distance on T⁷.
    
    On each S¹: d(θ₁, θ₂) = min(|θ₁-θ₂|, 2π - |θ₁-θ₂|)
    Total: D = sqrt(Σᵢ dᵢ²)
    """
    N = angles.shape[0]
    
    # Difference in each coordinate: (N, N, 7)
    diff = np.abs(angles[:, None, :] - angles[None, :, :])
    
    # Toric distance on each circle
    diff_toric = np.minimum(diff, 2 * np.pi - diff)
    
    # Euclidean combination
    D = np.sqrt(np.sum(diff_toric**2, axis=2))
    
    return D.astype(np.float32)


# Quick test
print("Testing T⁷ sampling...")
test_angles = sample_T7(100, seed=42)
print(f"  Shape: {test_angles.shape}")
print(f"  Range: [{test_angles.min():.4f}, {test_angles.max():.4f}]")

test_D = geodesic_distance_T7(test_angles)
print(f"  Distance range: [{test_D.min():.4f}, {test_D.max():.4f}]")
print(f"  Expected max: √7 × π ≈ {np.sqrt(7) * np.pi:.4f}")
print("✓ T⁷ functions OK")

In [ ]:
# Cell 4: Graph Laplacian (IDENTICAL to K₇ pipeline)

def compute_lambda1(D: np.ndarray, k: int, use_gpu: bool = True) -> tuple:
    """
    Compute first non-zero eigenvalue of normalized Laplacian.
    
    THIS IS THE EXACT SAME PIPELINE AS K₇:
    1. Adaptive bandwidth: σ = median(k-NN distances)
    2. Gaussian kernel: W = exp(-D²/2σ²)
    3. k-NN sparsification
    4. Symmetric normalization: L = I - D^(-1/2) W D^(-1/2)
    5. Second smallest eigenvalue
    
    Returns: (lambda1, sigma)
    """
    N = D.shape[0]
    k = min(k, N - 1)
    
    # Adaptive bandwidth from k-NN (same as K₇)
    knn_dists = np.partition(D, k+1, axis=1)[:, k]
    sigma = float(np.median(knn_dists))
    
    # Gaussian kernel
    W = np.exp(-D**2 / (2 * sigma**2))
    np.fill_diagonal(W, 0)
    
    # k-NN sparsification
    for i in range(N):
        idx = np.argpartition(D[i], k+1)[:k+1]
        mask = np.ones(N, dtype=bool)
        mask[idx] = False
        W[i, mask] = 0
    
    # Symmetrize
    W = (W + W.T) / 2
    
    # Normalized Laplacian
    deg = np.maximum(W.sum(axis=1), 1e-10)
    D_inv_sqrt = 1.0 / np.sqrt(deg)
    L = np.eye(N) - np.outer(D_inv_sqrt, D_inv_sqrt) * W
    
    # Compute eigenvalues
    if use_gpu and GPU_AVAILABLE:
        L_gpu = cp_csr(cp.array(L, dtype=cp.float32))
        eigs, _ = cp_eigsh(L_gpu, k=5, which='SA')
        eigs = cp.asnumpy(eigs)
    else:
        from scipy.sparse import csr_matrix as sp_csr
        from scipy.sparse.linalg import eigsh as sp_eigsh
        L_sparse = sp_csr(L)
        eigs, _ = sp_eigsh(L_sparse, k=5, which='SM')
    
    eigs = np.sort(np.real(eigs))
    
    # First non-zero eigenvalue
    for ev in eigs:
        if ev > 1e-8:
            return float(ev), sigma
    
    return float(eigs[1]) if len(eigs) > 1 else 0.0, sigma

print("✓ Graph Laplacian function defined (identical to K₇ pipeline)")

In [ ]:
# Cell 5: Calibration Study Configuration

# Match K₇ protocol exactly
# K₇ used: k = 0.74 × √N (empirical scaling)

SCALING_COEFF = 0.74  # Same as K₇

# N values (same range as K₇ study)
N_VALUES = [5000, 10000, 20000, 30000, 50000]

# We also test theoretical scaling for comparison
# For d=7: k ~ N^(2/(d+4)) = N^(2/11) ≈ N^0.18
# For d=3: k ~ N^(2/(d+4)) = N^(2/7) ≈ N^0.29

ALPHA_EMPIRICAL = 0.5    # √N scaling (what K₇ used)
ALPHA_THEORY_D7 = 2/11   # Theoretical for d=7
ALPHA_THEORY_D3 = 2/7    # Theoretical for d=3

print("Calibration Configuration:")
print(f"  N values: {N_VALUES}")
print(f"  Empirical scaling: k = {SCALING_COEFF} × N^{ALPHA_EMPIRICAL}")
print(f"  Theoretical (d=7): k ~ N^{ALPHA_THEORY_D7:.4f}")
print(f"  Theoretical (d=3): k ~ N^{ALPHA_THEORY_D3:.4f}")

In [ ]:
# Cell 6: S³ Calibration Study

print("=" * 70)
print("S³ CALIBRATION (λ₁ exact = 3.0)")
print("=" * 70)
print(f"\nScaling: k = {SCALING_COEFF} × √N (same as K₇)")
print()

s3_results = []

print(f"{'N':>7} | {'k':>5} | {'λ₁ measured':>12} | {'λ₁ exact':>10} | {'Bias':>10} | {'Bias %':>8}")
print("-" * 70)

for N in N_VALUES:
    # Same k-scaling as K₇
    k = max(15, int(SCALING_COEFF * np.sqrt(N)))
    
    t0 = time.time()
    
    # Sample S³
    points = sample_S3(N, seed=42)
    
    # Compute distances
    D = geodesic_distance_S3(points)
    
    # Compute λ₁
    lambda1, sigma = compute_lambda1(D, k)
    
    elapsed = time.time() - t0
    
    # Compute bias
    bias = lambda1 - LAMBDA1_S3
    bias_pct = bias / LAMBDA1_S3 * 100
    
    print(f"{N:>7} | {k:>5} | {lambda1:>12.6f} | {LAMBDA1_S3:>10.1f} | {bias:>+10.4f} | {bias_pct:>+7.2f}%")
    
    s3_results.append({
        'N': N,
        'k': k,
        'lambda1': float(lambda1),
        'lambda1_exact': LAMBDA1_S3,
        'bias': float(bias),
        'bias_pct': float(bias_pct),
        'sigma': float(sigma),
        'time_s': float(elapsed)
    })
    
    # Cleanup
    del points, D
    clear_gpu()

# Summary
mean_bias = np.mean([r['bias'] for r in s3_results[-3:]])  # Last 3 (high N)
mean_bias_pct = np.mean([r['bias_pct'] for r in s3_results[-3:]])

print("\n" + "-" * 70)
print(f"S³ SUMMARY (high N):")
print(f"  Mean bias: {mean_bias:+.4f}")
print(f"  Mean bias %: {mean_bias_pct:+.2f}%")
print(f"  If K₇ shows similar bias, then 13 vs 14 is discretization artifact.")

In [ ]:
# Cell 7: T⁷ Calibration Study

print("=" * 70)
print("T⁷ CALIBRATION (λ₁ exact = 1.0, same dimension as K₇!)")
print("=" * 70)
print(f"\nScaling: k = {SCALING_COEFF} × √N (same as K₇)")
print()

t7_results = []

print(f"{'N':>7} | {'k':>5} | {'λ₁ measured':>12} | {'λ₁ exact':>10} | {'Bias':>10} | {'Bias %':>8}")
print("-" * 70)

for N in N_VALUES:
    # Same k-scaling as K₇
    k = max(15, int(SCALING_COEFF * np.sqrt(N)))
    
    t0 = time.time()
    
    # Sample T⁷
    angles = sample_T7(N, seed=42)
    
    # Compute distances
    D = geodesic_distance_T7(angles)
    
    # Compute λ₁
    lambda1, sigma = compute_lambda1(D, k)
    
    elapsed = time.time() - t0
    
    # Compute bias
    bias = lambda1 - LAMBDA1_T7
    bias_pct = bias / LAMBDA1_T7 * 100
    
    print(f"{N:>7} | {k:>5} | {lambda1:>12.6f} | {LAMBDA1_T7:>10.1f} | {bias:>+10.4f} | {bias_pct:>+7.2f}%")
    
    t7_results.append({
        'N': N,
        'k': k,
        'lambda1': float(lambda1),
        'lambda1_exact': LAMBDA1_T7,
        'bias': float(bias),
        'bias_pct': float(bias_pct),
        'sigma': float(sigma),
        'time_s': float(elapsed)
    })
    
    # Cleanup
    del angles, D
    clear_gpu()

# Summary
mean_bias_t7 = np.mean([r['bias'] for r in t7_results[-3:]])
mean_bias_pct_t7 = np.mean([r['bias_pct'] for r in t7_results[-3:]])

print("\n" + "-" * 70)
print(f"T⁷ SUMMARY (high N):")
print(f"  Mean bias: {mean_bias_t7:+.4f}")
print(f"  Mean bias %: {mean_bias_pct_t7:+.2f}%")
print(f"  T⁷ has SAME DIMENSION as K₇ - this is the key calibration!")

In [ ]:
# Cell 8: Stability Test (multiple seeds)

print("=" * 70)
print("STABILITY TEST: Multiple seeds at N=30000")
print("=" * 70)

N_STABILITY = 30000
k_stability = max(15, int(SCALING_COEFF * np.sqrt(N_STABILITY)))
N_SEEDS = 5

print(f"\nN = {N_STABILITY}, k = {k_stability}")
print()

# S³ stability
print("S³ stability:")
s3_stability = []
for seed in range(N_SEEDS):
    points = sample_S3(N_STABILITY, seed=seed)
    D = geodesic_distance_S3(points)
    lambda1, _ = compute_lambda1(D, k_stability)
    bias = lambda1 - LAMBDA1_S3
    print(f"  Seed {seed}: λ₁ = {lambda1:.6f}, bias = {bias:+.4f}")
    s3_stability.append({'seed': seed, 'lambda1': float(lambda1), 'bias': float(bias)})
    del points, D
    clear_gpu()

s3_mean = np.mean([r['lambda1'] for r in s3_stability])
s3_std = np.std([r['lambda1'] for r in s3_stability])
print(f"  Mean: {s3_mean:.6f} ± {s3_std:.6f}")

# T⁷ stability
print("\nT⁷ stability:")
t7_stability = []
for seed in range(N_SEEDS):
    angles = sample_T7(N_STABILITY, seed=seed)
    D = geodesic_distance_T7(angles)
    lambda1, _ = compute_lambda1(D, k_stability)
    bias = lambda1 - LAMBDA1_T7
    print(f"  Seed {seed}: λ₁ = {lambda1:.6f}, bias = {bias:+.4f}")
    t7_stability.append({'seed': seed, 'lambda1': float(lambda1), 'bias': float(bias)})
    del angles, D
    clear_gpu()

t7_mean = np.mean([r['lambda1'] for r in t7_stability])
t7_std = np.std([r['lambda1'] for r in t7_stability])
print(f"  Mean: {t7_mean:.6f} ± {t7_std:.6f}")

In [ ]:
# Cell 9: Calibration Factor Computation

print("=" * 70)
print("CALIBRATION FACTOR ANALYSIS")
print("=" * 70)

# At high N, what is the multiplicative bias?
# If measured = factor × exact, then factor = measured / exact

# S³ factor
s3_factor = s3_mean / LAMBDA1_S3
print(f"\nS³ calibration factor: {s3_factor:.6f}")
print(f"  (measured λ₁ = {s3_factor:.4f} × exact λ₁)")

# T⁷ factor
t7_factor = t7_mean / LAMBDA1_T7
print(f"\nT⁷ calibration factor: {t7_factor:.6f}")
print(f"  (measured λ₁ = {t7_factor:.4f} × exact λ₁)")

# Average factor (for dimension 7, use T⁷)
print("\n" + "-" * 40)
print("KEY INSIGHT:")
print(f"  The pipeline has a bias factor of ~{t7_factor:.4f} on dimension 7.")
print(f"  This means measured values are {(t7_factor-1)*100:+.1f}% off from exact.")

# Apply to K₇
print("\n" + "-" * 40)
print("APPLICATION TO K₇:")
K7_MEASURED = 13.07 / H_STAR_K7  # From K7_SPECTRAL_FINAL_SYNTHESIS.md
K7_CORRECTED = K7_MEASURED / t7_factor
K7_CORRECTED_PRODUCT = K7_CORRECTED * H_STAR_K7

print(f"  K₇ measured: λ₁ × H* = 13.07")
print(f"  T⁷ calibration factor: {t7_factor:.4f}")
print(f"  K₇ corrected: λ₁ × H* = 13.07 / {t7_factor:.4f} = {K7_CORRECTED_PRODUCT:.2f}")
print(f"\n  If corrected ≈ 14: the −1 is discretization bias")
print(f"  If corrected ≈ 13: the 13 is structural (dim(G₂) - h)")

In [ ]:
# Cell 10: Visualization

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: S³ convergence
ax1 = axes[0, 0]
Ns = [r['N'] for r in s3_results]
l1s = [r['lambda1'] for r in s3_results]
ax1.plot(Ns, l1s, 'o-', color='blue', linewidth=2, markersize=8, label='Measured')
ax1.axhline(y=LAMBDA1_S3, color='red', linestyle='--', linewidth=2, label=f'Exact = {LAMBDA1_S3}')
ax1.fill_between(Ns, LAMBDA1_S3 * 0.95, LAMBDA1_S3 * 1.05, alpha=0.2, color='green', label='±5%')
ax1.set_xlabel('N', fontsize=12)
ax1.set_ylabel('λ₁', fontsize=12)
ax1.set_title('S³ Calibration (λ₁ exact = 3)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: T⁷ convergence
ax2 = axes[0, 1]
Ns = [r['N'] for r in t7_results]
l1s = [r['lambda1'] for r in t7_results]
ax2.plot(Ns, l1s, 'o-', color='green', linewidth=2, markersize=8, label='Measured')
ax2.axhline(y=LAMBDA1_T7, color='red', linestyle='--', linewidth=2, label=f'Exact = {LAMBDA1_T7}')
ax2.fill_between(Ns, LAMBDA1_T7 * 0.95, LAMBDA1_T7 * 1.05, alpha=0.2, color='blue', label='±5%')
ax2.set_xlabel('N', fontsize=12)
ax2.set_ylabel('λ₁', fontsize=12)
ax2.set_title('T⁷ Calibration (λ₁ exact = 1, dim=7 like K₇)', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Bias comparison
ax3 = axes[1, 0]
s3_biases = [r['bias_pct'] for r in s3_results]
t7_biases = [r['bias_pct'] for r in t7_results]
Ns = [r['N'] for r in s3_results]
ax3.plot(Ns, s3_biases, 'o-', color='blue', linewidth=2, markersize=8, label='S³ bias')
ax3.plot(Ns, t7_biases, 's-', color='green', linewidth=2, markersize=8, label='T⁷ bias')
ax3.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
ax3.axhline(y=-7.14, color='red', linestyle='--', alpha=0.7, label='−7.14% (13→14)')
ax3.set_xlabel('N', fontsize=12)
ax3.set_ylabel('Bias (%)', fontsize=12)
ax3.set_title('Bias vs N', fontsize=12)
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Summary bars
ax4 = axes[1, 1]
categories = ['S³ measured', 'S³ exact', 'T⁷ measured', 'T⁷ exact', 'K₇ measured', 'K₇ corrected']
values = [s3_mean, LAMBDA1_S3, t7_mean, LAMBDA1_T7, 13.07, K7_CORRECTED_PRODUCT]
colors = ['lightblue', 'blue', 'lightgreen', 'green', 'orange', 'red']
bars = ax4.bar(categories, values, color=colors, alpha=0.7)

for bar, val in zip(bars, values):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 0.2,
             f'{val:.2f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

ax4.axhline(y=14, color='purple', linestyle='--', alpha=0.5, label='14 (Pell)')
ax4.axhline(y=13, color='brown', linestyle='--', alpha=0.5, label='13 (numerical)')
ax4.set_ylabel('λ₁ (or λ₁×H*)', fontsize=12)
ax4.set_title('Summary Comparison', fontsize=12)
ax4.legend()
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.savefig('Spectral_Calibration_S3_T7.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Saved: Spectral_Calibration_S3_T7.png")

In [ ]:
# Cell 11: Final Conclusions

print("=" * 70)
print("FINAL CONCLUSIONS")
print("=" * 70)

# Decision logic
THRESHOLD_STRUCTURAL = 0.05  # 5% - if bias is small, 13 is structural
THRESHOLD_ARTIFACT = 0.07   # 7% - if bias matches 13→14 correction needed

t7_bias_magnitude = abs(t7_factor - 1)
needed_correction = 14/13 - 1  # ~7.7%

print(f"\n1. CALIBRATION RESULTS:")
print(f"   S³: measured/exact = {s3_factor:.4f} (bias = {(s3_factor-1)*100:+.2f}%)")
print(f"   T⁷: measured/exact = {t7_factor:.4f} (bias = {(t7_factor-1)*100:+.2f}%)")

print(f"\n2. K₇ INTERPRETATION:")
print(f"   Measured: λ₁×H* = 13.07")
print(f"   Correction needed for 14: {needed_correction*100:.2f}%")
print(f"   Pipeline bias (T⁷): {t7_bias_magnitude*100:.2f}%")

print(f"\n3. VERDICT:")
if abs(t7_bias_magnitude - needed_correction) < 0.02:  # Within 2%
    print(f"   ⚠️ BIAS MATCHES CORRECTION")
    print(f"   The −1 (13 vs 14) is likely a DISCRETIZATION ARTIFACT.")
    print(f"   True value: λ₁×H* = 14 (Pell equation)")
    verdict = "ARTIFACT"
elif t7_bias_magnitude < 0.03:  # Less than 3%
    print(f"   ✓ BIAS IS SMALL")
    print(f"   The 13 is likely STRUCTURAL (dim(G₂) - h = 14 - 1).")
    print(f"   True value: λ₁×H* = 13")
    verdict = "STRUCTURAL"
else:
    print(f"   ❓ INCONCLUSIVE")
    print(f"   Need larger N or better method.")
    verdict = "INCONCLUSIVE"

print(f"\n" + "=" * 70)

In [ ]:
# Cell 12: Save Results

output = {
    "metadata": {
        "notebook": "Spectral_Calibration_S3_T7.ipynb",
        "timestamp": datetime.now().isoformat(),
        "purpose": "Calibrate spectral pipeline on spaces with known λ₁",
        "scaling": f"k = {SCALING_COEFF} × sqrt(N)",
        "gpu": GPU_AVAILABLE
    },
    "exact_eigenvalues": {
        "S3": LAMBDA1_S3,
        "T7": LAMBDA1_T7
    },
    "s3_results": s3_results,
    "t7_results": t7_results,
    "s3_stability": s3_stability,
    "t7_stability": t7_stability,
    "calibration_factors": {
        "S3": float(s3_factor),
        "T7": float(t7_factor),
        "S3_mean": float(s3_mean),
        "S3_std": float(s3_std),
        "T7_mean": float(t7_mean),
        "T7_std": float(t7_std)
    },
    "k7_analysis": {
        "measured_product": 13.07,
        "corrected_product": float(K7_CORRECTED_PRODUCT),
        "correction_factor": float(t7_factor),
        "verdict": verdict
    },
    "conclusions": {
        "t7_bias_pct": float((t7_factor - 1) * 100),
        "needed_for_14_pct": float(needed_correction * 100),
        "verdict": verdict,
        "interpretation": (
            "13 is structural (dim(G₂) - h)" if verdict == "STRUCTURAL" else
            "13 is discretization artifact, true value is 14" if verdict == "ARTIFACT" else
            "Inconclusive, need more data"
        )
    }
}

filename = "Spectral_Calibration_S3_T7_results.json"
with open(filename, "w") as f:
    json.dump(output, f, indent=2)

print(f"\n✓ Saved: {filename}")
print("\nDownload this file and the PNG to share with Claude!")

---

## Next Steps

Based on results:

### If verdict = STRUCTURAL:
- The 13 is real: λ₁ × H* = dim(G₂) - h = 14 - 1 = 13
- Focus on **proving** this analytically (Cheeger bounds, index theory)
- The Pell equation 99² - 50×14² = 1 might encode the relation differently

### If verdict = ARTIFACT:
- The true value is λ₁ × H* = 14 (matches Pell)
- Use calibration factor to correct future measurements
- Focus on improving numerical methods for better convergence

### If verdict = INCONCLUSIVE:
- Run with larger N (75k, 100k)
- Try Belkin-Niyogi scaling k ~ N^(6/13)
- Consider Cheeger analytical bounds as alternative

---

*GIFT Spectral Gap Research Program — Calibration Study*